# Single-Cell RNA-seq Analysis Pipeline

Interactive pipeline for scRNA-seq preprocessing, dimensionality reduction, clustering, and annotation.

## Pipeline Overview

1. **Data Loading** - Load h5ad, rds, or mtx format
2. **Quality Control** - Interactive QC metric visualization and filtering
3. **Doublet Detection** - Identify and remove doublets
4. **Normalization** - Choose normalization method
5. **Feature Selection** - Select highly variable genes
6. **Dimensionality Reduction** - PCA and UMAP
7. **Clustering** - Interactive resolution selection
8. **Gene Visualization** - View gene expression on UMAP
9. **Marker Gene Analysis** - Identify cluster markers
10. **Manual Annotation** - Annotate cell types
11. **Subclustering** (optional) - Refine specific clusters
12. **Export & Report** - Save results and generate PDF report

## Setup

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import pipeline modules
from src import io, qc, preprocessing, reduction, clustering, annotation, visualization, interactive, reports

# Scanpy settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

print("✓ Setup complete")

## 1. Data Loading

Load your scRNA-seq data. Supported formats:
- `.h5ad` (AnnData)
- `.rds` (Seurat object)
- `matrix.mtx` + `barcodes.tsv` + `genes.tsv` (10X format)

In [ ]:
# Uses: io.load_h5ad() / io.load_rds() / io.load_10x_mtx() / io.load_data()

# Option 1: Load h5ad file
# adata = io.load_h5ad('../data/your_data.h5ad')

# Option 2: Load RDS file (Seurat object)
# adata = io.load_rds('../data/your_data.rds')

# Option 3: Load 10X MTX format
# adata = io.load_10x_mtx('../data/filtered_feature_bc_matrix/')

# Option 4: Auto-detect format
adata = io.load_data('../data/your_data.h5ad')  # Change to your file path

print(f"\nLoaded: {adata.n_obs} cells × {adata.n_vars} genes")

In [ ]:
# Uses: io.save_checkpoint()

# Save raw data checkpoint
io.save_checkpoint(adata, 'raw')
print("✓ Raw data saved")

## 2. Quality Control

Calculate QC metrics and visualize data quality.

In [ ]:
# Uses: qc.calculate_qc_metrics()
# Replaces old wrapper: qc.get_qc_summary() → use adata.obs[metrics].describe()

# Calculate QC metrics
adata = qc.calculate_qc_metrics(adata, mito_prefix='MT-')

# Display QC summary (direct pandas operation)
metrics = ['total_counts', 'n_genes_by_counts', 'pct_counts_mt']
if 'pct_counts_ribo' in adata.obs.columns:
    metrics.append('pct_counts_ribo')
summary = adata.obs[metrics].describe().T
print("\nQC Metrics Summary:")
display(summary)

In [ ]:
# Uses: qc.plot_qc_metrics()

# Generate QC plots
fig = qc.plot_qc_metrics(adata, figsize=(15, 10))
plt.show()

### Interactive QC Filtering

Adjust thresholds interactively and see the number of cells that will be retained.

In [ ]:
# Uses: interactive.QCFilterWidget() which uses qc.calculate_mad_thresholds(), qc.plot_qc_metrics()
# Replaces old wrapper: qc.filter_cells() → now uses direct filtering in widget

# Interactive QC filtering widget
qc_widget = interactive.QCFilterWidget(adata)
qc_widget.display()

In [ ]:
# Get filtered data from widget (uses: sc.pp.filter_genes() internally)

# Apply filters (run after clicking "Apply Filters" button)
adata = qc_widget.get_filtered_data()

if adata is not None:
    print(f"\n✓ Filtered data: {adata.n_obs} cells × {adata.n_vars} genes")
else:
    print("Please click 'Apply Filters' button first")

In [ ]:
# Replaces old wrapper: qc.filter_genes_by_counts() → use sc.pp.filter_genes() directly

# Filter genes (remove genes detected in < 20 cells)
n_genes_before = adata.n_vars
sc.pp.filter_genes(adata, min_cells=20)
n_genes_after = adata.n_vars
print(f"Genes: {n_genes_before} → {n_genes_after} (removed {n_genes_before - n_genes_after})")

## 3. Doublet Detection

Identify and remove potential doublets using Scrublet.

In [ ]:
# Uses: preprocessing.detect_doublets() (wraps Scrublet library)

# Detect doublets
adata = preprocessing.detect_doublets(adata, expected_doublet_rate=0.06)

### Interactive Doublet Filtering

In [ ]:
# Uses: interactive.DoubletFilterWidget() which uses preprocessing.plot_doublet_scores()
# Replaces old wrapper: preprocessing.filter_doublets() → now uses direct filtering in widget

# Interactive doublet filtering widget
doublet_widget = interactive.DoubletFilterWidget(adata)
doublet_widget.display()

In [ ]:
# Get filtered data from widget

# Apply doublet filtering (run after clicking "Remove Doublets" button)
adata = doublet_widget.get_filtered_data()

if adata is not None:
    print(f"\n✓ After doublet removal: {adata.n_obs} cells × {adata.n_vars} genes")
else:
    print("Please click 'Remove Doublets' button first")

## 4. Normalization

Choose normalization method based on your analysis goals:
- **log1p**: Standard log transformation (recommended for general use)
- **scran**: Better for batch correction
- **pearson**: Better for rare cell type identification

In [ ]:
# Replaces old wrapper: preprocessing.normalize_data() → use Scanpy directly

# Store raw counts
adata.layers['counts'] = adata.X.copy()

# Normalize data (log1p method)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print("✓ Normalization complete (log1p)")

## 5. Feature Selection

Select highly variable genes for downstream analysis.

In [ ]:
# Replaces old wrappers:
#   - preprocessing.select_highly_variable_genes() → use sc.pp.highly_variable_genes()
#   - preprocessing.plot_highly_variable_genes() → use sc.pl.highly_variable_genes()

# Select highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor='seurat')
print(f"Selected {adata.var['highly_variable'].sum()} highly variable genes")

# Plot HVGs
sc.pl.highly_variable_genes(adata)

## 6. Scaling and Regression (Optional)

Scale data and optionally regress out unwanted sources of variation.

In [ ]:
# Replaces old wrappers:
#   - preprocessing.regress_out() → use sc.pp.regress_out()
#   - preprocessing.scale_data() → use sc.pp.scale()

# Optional: Regress out total counts and mitochondrial percentage
# sc.pp.regress_out(adata, keys=['total_counts', 'pct_counts_mt'])

# Scale data
sc.pp.scale(adata, max_value=10)
print("✓ Data scaled")

## 7. Dimensionality Reduction

Compute PCA, then UMAP for visualization.

In [ ]:
# Replaces old wrapper: reduction.compute_pca() → use sc.tl.pca()
# Uses: reduction.plot_pca_variance() for custom variance plots

# Compute PCA
sc.tl.pca(adata, n_comps=50, use_highly_variable=True)
print(f"PCA computed: {adata.obsm['X_pca'].shape}")

# Plot variance explained (custom plot)
fig = reduction.plot_pca_variance(adata, n_pcs=50)
plt.show()

### Interactive PCA Component Selection

In [ ]:
# Uses: interactive.PCAWidget() which uses reduction.plot_pca_variance()

# Interactive PCA selection widget
pca_widget = interactive.PCAWidget(adata)
pca_widget.display()

In [ ]:
# Get selected number of PCs from widget

# Get selected number of PCs
n_pcs = pca_widget.get_n_pcs()
print(f"Using {n_pcs} principal components")

In [ ]:
# Replaces old wrappers: reduction.compute_neighbors(), reduction.compute_umap() → use Scanpy directly

# Compute neighbors
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=n_pcs)
print("Neighbors computed")

# Compute UMAP
sc.tl.umap(adata, min_dist=0.5)
print("✓ UMAP computed")

## 8. Clustering

Test multiple resolutions and select the optimal clustering.

In [ ]:
# Uses: clustering.compute_multiple_resolutions() (batch Leiden clustering)

# Compute clustering at multiple resolutions
clustering.compute_multiple_resolutions(adata, resolutions=[0.25, 0.5, 1.0, 1.5, 2.0])

### Interactive Resolution Selection

In [ ]:
# Uses: interactive.ClusteringResolutionWidget() which uses sc.tl.leiden(), sc.pl.umap()

# Interactive clustering widget
clustering_widget = interactive.ClusteringResolutionWidget(adata, resolutions=[0.25, 0.5, 1.0, 1.5, 2.0])
clustering_widget.display()

In [ ]:
# Replaces old wrapper: reduction.plot_umap() → use sc.pl.umap()

# Get selected resolution
selected_res = clustering_widget.get_resolution()
print(f"\nUsing resolution: {selected_res}")

# Plot selected clustering
sc.pl.umap(adata, color='leiden', legend_loc='on data', frameon=False)

In [ ]:
# Uses: clustering.plot_cluster_sizes(), clustering.compute_cluster_statistics()

# Display cluster sizes
fig = clustering.plot_cluster_sizes(adata, clustering_key='leiden')
plt.show()

# Cluster statistics
stats = clustering.compute_cluster_statistics(adata, clustering_key='leiden')
display(stats)

## 9. Save Preprocessed Data

Save checkpoint before annotation.

In [ ]:
# Uses: io.save_checkpoint()

# Save preprocessed data
io.save_checkpoint(adata, 'preprocessed')
print("✓ Preprocessed data saved")

## 10. Gene Expression Visualization

Visualize specific genes to inform annotation decisions.

In [ ]:
# Uses: interactive.GeneVisualizationWidget() which uses sc.pl.umap()

# Interactive gene visualization widget
gene_widget = interactive.GeneVisualizationWidget(adata)
gene_widget.display()

# Enter gene names (one per line) and click "Plot Genes"

## 11. Marker Gene Analysis

Identify marker genes for each cluster.

In [ ]:
# Replaces old wrapper: annotation.find_marker_genes() → use sc.tl.rank_genes_groups()

# Find marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden', method='wilcoxon', n_genes=100)
print("✓ Marker genes identified")

In [ ]:
# Replaces old wrapper: annotation.plot_marker_genes_heatmap() → use sc.pl.rank_genes_groups_heatmap()

# Plot top marker genes
sc.pl.rank_genes_groups_heatmap(adata, n_genes=10, groupby='leiden')

In [ ]:
# Replaces old wrapper: annotation.plot_marker_genes_dotplot() → use sc.pl.rank_genes_groups_dotplot()

# Dotplot of top markers
sc.pl.rank_genes_groups_dotplot(adata, n_genes=5, groupby='leiden')

In [ ]:
# Uses: annotation.get_top_markers() (extracts data from sc.tl.rank_genes_groups results)

# View top markers for specific cluster
cluster_id = 0  # Change to cluster of interest
markers = annotation.get_top_markers(adata, cluster=cluster_id, n_genes=20)
display(markers)

## 12. Manual Cell Type Annotation

Annotate clusters based on marker genes.

In [ ]:
# Uses: interactive.AnnotationWidget() which uses annotation.get_top_markers(), annotation.annotate_clusters(), sc.pl.umap()

# Interactive annotation widget
annotation_widget = interactive.AnnotationWidget(adata, clustering_key='leiden')
annotation_widget.display()

# For each cluster:
# 1. Select cluster from dropdown
# 2. Click "Show Markers" to see top marker genes
# 3. Enter cell type in text box
# 4. Click "Annotate Cluster"
# 5. Repeat for all clusters
# 6. Click "Finish Annotation" when done

### Alternative: Manual Annotation (Code)

If you prefer to annotate programmatically:

In [ ]:
# Uses: annotation.annotate_clusters() (maps cluster IDs to cell type names)

# Manual annotation dictionary
# annotations = {
#     '0': 'T cells',
#     '1': 'B cells',
#     '2': 'Monocytes',
#     '3': 'NK cells',
#     # ... add all clusters
# }

# annotation.annotate_clusters(adata, annotations=annotations, clustering_key='leiden')

In [ ]:
# Replaces old wrapper: annotation.plot_annotated_umap() → use sc.pl.umap()

# Plot annotated UMAP
if 'cell_type' in adata.obs.columns:
    sc.pl.umap(adata, color='cell_type', legend_loc='right margin', frameon=False)

## 13. Cluster Manipulation (Optional)

Split or merge clusters if needed.

In [ ]:
# Uses: clustering.merge_clusters(), clustering.split_cluster()

# Merge clusters (example: merge clusters 3 and 5)
# clustering.merge_clusters(adata, clusters_to_merge=['3', '5'], clustering_key='leiden', new_cluster_name='3')

# Split cluster (example: split cluster 2)
# clustering.split_cluster(adata, cluster_id='2', clustering_key='leiden', resolution=1.5)

## 14. Subclustering (Optional)

Perform detailed analysis on specific cell populations.

In [ ]:
# Uses: clustering.subcluster_cells() (full subclustering pipeline with HVG, PCA, UMAP, Leiden)

# Subcluster specific clusters
# cluster_ids = ['0', '1']  # Specify clusters to subcluster
# adata_subset = clustering.subcluster_cells(
#     adata,
#     cluster_ids=cluster_ids,
#     clustering_key='leiden',
#     resolution=1.0
# )

# # Visualize subclusters (uses sc.pl.umap)
# sc.pl.umap(adata_subset, color='leiden_sub', legend_loc='on data', frameon=False)

# # Find markers for subclusters (uses sc.tl.rank_genes_groups)
# sc.tl.rank_genes_groups(adata_subset, groupby='leiden_sub', method='wilcoxon')

## 15. Save Annotated Data

In [ ]:
# Uses: io.save_checkpoint()
# Replaces old wrapper: annotation.export_annotations() → use adata.obs.to_csv()

# Save final annotated data
io.save_checkpoint(adata, 'annotated')
print("✓ Annotated data saved")

# Export annotations to CSV
if 'cell_type' in adata.obs.columns:
    adata.obs[['cell_type']].to_csv('../outputs/cell_annotations.csv')
    print("✓ Annotations exported to CSV")

## 16. Generate Reports

In [ ]:
# Uses: reports.create_qc_report() which uses qc.plot_qc_metrics(), preprocessing.plot_doublet_scores(), visualization.plot_qc_overview()

# Generate QC report
reports.create_qc_report(adata, output_file='../outputs/reports/qc_report.pdf')

In [ ]:
# Uses: reports.create_analysis_report() which uses:
#   - visualization.create_summary_figure()
#   - reduction.plot_pca_variance()
#   - clustering.plot_cluster_sizes(), clustering.compute_cluster_statistics()
#   - sc.pl.rank_genes_groups_heatmap(), sc.pl.rank_genes_groups_dotplot(), sc.pl.umap()

# Generate analysis report
reports.create_analysis_report(
    adata,
    output_file='../outputs/reports/analysis_report.pdf',
    clustering_key='leiden',
    annotation_key='cell_type'
)

## Summary

Pipeline complete! Your outputs are saved in:
- **Checkpoints**: `../outputs/checkpoints/`
  - `raw.h5ad`
  - `preprocessed.h5ad`
  - `annotated.h5ad`
- **Reports**: `../outputs/reports/`
  - `qc_report.pdf`
  - `analysis_report.pdf`
- **Annotations**: `../outputs/cell_annotations.csv`